# Synthetic generation
Notebook used to make whole stacks of synthetic data based on single step results (synthetic_test.ipynb).

In [1]:
%matplotlib inline

import os, time
import warnings
import math
import ipywidgets as widgets
from ipywidgets import interact

import numpy as np
import matplotlib.pyplot as plt
from skimage import io, draw
from scipy.stats import multivariate_normal
from imgaug import augmenters as iaa

from utils_common.image import to_npint
from utils_common.processing import flood_fill

%load_ext autoreload
%autoreload 2

In [2]:
shape = (192, 256)

# All in one
Function to create a synthetic image/stack from scratch.

In [8]:
def synthetic_stack(shape, n_images, n_neurons=-1):
    """
    Return a stack of synthetic neural images.
    
    Args:
        shape: tuple of int
            Tuple (height, width) representing the shape of the images.
        n_images: int
            Number of images in the stack.
        n_neurons: int, default to -1
            Number of neurons to be present on the stack.
            If -1, will be randomly sampled.
            
    Returns:
        synth_stack: ndarray of shape NxHxW
            Stack of N synthetic images.
        synth_seg: ndarray of shape NxHxW
            Stack of N synthetic segmentations.
    """ 
    # Initialization
    n_samples = 1000 # number of samples for gaussian neurons
    grid_size = 8 # for the elastic deformation
    if n_neurons == -1:
        n_neurons = np.random.randint(2, 4 + 1)
    
    ## Create the gaussians representing the neurons
    max_neurons = []
    neurons = np.zeros((n_neurons,) + shape)
    neurons_segs = np.zeros((n_neurons,) + shape, dtype=np.bool)
    # Meshgrid for the gaussian weights
    rows, cols = np.arange(shape[0]), np.arange(shape[1])
    meshgrid = np.zeros(shape + (2,))
    meshgrid[:,:,0], meshgrid[:,:,1] = np.meshgrid(cols, rows) # note the order!
    for i in range(n_neurons):
        # Create neuron infinitly until in image and no overlap with another
        # TODO: change this to something that cannot loop to infinity
        while True:
            # Note that x and y axes are col and row (so, inversed!)
            mean = np.array([np.random.randint(shape[1]), np.random.randint(shape[0])])
            cross_corr = np.random.randint(-15, 15)
            cov = np.array([
                [np.random.randint(5, 15), cross_corr],
                [cross_corr, np.random.randint(50, 150)]
            ])

            # Bounding ellipses
            val, vec = np.linalg.eig(cov)
            rotation = math.atan2(vec[0, np.argmax(val)], vec[1, np.argmax(val)])
            rr, cc = draw.ellipse(mean[1], mean[0], 
                                  2*np.sqrt(cov[1,1]), 2*np.sqrt(cov[0,0]),
                                  rotation=rotation)
            # Check if outside the image
            if (rr < 0).any() or (rr >= shape[0]).any() or (cc < 0).any() or (cc >= shape[1]).any():
                continue
            # Check if overlapping with any existing neuron
            elif (neurons_segs[:, rr, cc] == True).any():
                continue
            else:
                break
        neurons_segs[i, rr, cc] = True
        
        # Create gaussian weight image
        neurons[i,:,:] = multivariate_normal.pdf(meshgrid, mean, cov)
        neurons[i,:,:] /= neurons[i,:,:].sum()

        # Sample randomly the neuron maximum (with pre-computed values)
        if np.random.rand() < 0.2276730082246407:
            max_neurons.append(1.0)
        else:
            loc = 0.6625502112855037
            scale = 0.13925117610178622
            max_neurons.append(np.clip(np.random.normal(loc=loc, scale=scale), 0, 1))
        
    # Reduce segmentations to one image
    neurons_segs = neurons_segs.sum(axis=0)
    
    ## Warp neurons for each image to create the stack
    # Define warping sequence
    wrpseq = iaa.Sequential([
        iaa.PiecewiseAffine(scale=0.025, nb_rows=grid_size, nb_cols=grid_size)
    ])
    wrp_segs = np.zeros((n_images,) + shape, dtype=np.bool)
    wrp_neurons = np.zeros((n_images,) + shape, dtype=neurons.dtype)
    for i in range(n_images):
        # Set the warping to deterministic for warping both neurons and segmentation the same way
        seq_det = wrpseq.to_deterministic()
        
        ## Warp the neurons
        for j in range(n_neurons):
            # Warp gaussian defining it
            wrp_gaussian = seq_det.augment_image(neurons[j])
            wrp_gaussian /= wrp_gaussian.sum()
            # Sample from it
            x = np.random.choice(shape[0] * shape[1], size=n_samples, p=wrp_gaussian.ravel())
            y, x = np.unravel_index(x, shape)
            hist = plt.hist2d(x, y, bins=[shape[1], shape[0]], range=[[0, shape[1]], [0, shape[0]]])
            plt.close()
            wrp_neurons[i] = np.maximum(wrp_neurons[i], hist[0].T / hist[0].max() * max_neurons[j])
            
        ## Warp the segmentation
        wrp_segs[i] = seq_det.augment_image(neurons_segs)
        # Fill the possible holes in warped segmentation
        # TODO: Update this to work with a neuron present at the origin
        wrp_segs[i] = flood_fill(wrp_segs[i])
    
    ## Add noise (sampled from an exponential distribution)
    # Scale factor was pre-computed on real data
    noise = np.random.exponential(scale=0.041733140976778674, size=(n_images,) + shape)
    
    synth_stack = np.maximum(wrp_neurons, noise)
    synth_seg = wrp_segs
    return synth_stack, synth_seg

In [9]:
shape = (192, 256)
n_images = 25
n_neurons = 2
synth_stack, synth_seg = synthetic_stack(shape, n_images, n_neurons=n_neurons)

@interact(image = (0, n_images - 1))
def plot_data(image=0):
    plt.figure(figsize=(14,5))
    plt.subplot(121)
    plt.imshow(synth_stack[image], vmin=0, vmax=1, cmap='gray')
    plt.subplot(122)
    plt.imshow(synth_seg[image], cmap='gray')
    plt.show()

interactive(children=(IntSlider(value=0, description='image', max=24), Output()), _dom_classes=('widget-intera…

# Save synthetic stacks
Save some synthetic stacks in the dataset. Keep them in a separate folder "synthetic/".